In [1]:
import kotlin.io.path.Path

val keyDir = Path("./")

In [11]:
@file:OptIn(ExperimentalStdlibApi::class)

import java.io.File
import java.security.Key
import java.security.KeyFactory
import java.security.KeyPair
import java.security.KeyPairGenerator
import java.security.spec.KeySpec
import java.security.spec.NamedParameterSpec
import java.security.spec.PKCS8EncodedKeySpec
import java.security.spec.X509EncodedKeySpec
import javax.crypto.KEM
import kotlin.io.path.Path


private inline fun <reified T: KeySpec> KeyFactory.getKeySpec(key: Key): T =
    getKeySpec(key, T::class.java)


private val ML_KEM = "ML-KEM"
val keyDir = Path("./")

val f = KeyFactory.getInstance(ML_KEM)

val keys: Pair<File, File> = keyDir.resolve("key.pkcs8").toFile() to keyDir.resolve("key.pub").toFile()
// maybe monad
val kp: KeyPair = keys.takeIf { it.first.isFile && it.second.isFile }?.let { (privateKeyFile, pubKeyFile) ->
    val p8 = PKCS8EncodedKeySpec(privateKeyFile.readBytes(), ML_KEM)
    val privateKey = f.generatePrivate(p8)

    val x509 = X509EncodedKeySpec(pubKeyFile.readBytes(), ML_KEM)
    val pubKey = f.generatePublic(x509)

    KeyPair(pubKey, privateKey)
} ?: run {
    val kpg = KeyPairGenerator.getInstance(ML_KEM)
    kpg.initialize(NamedParameterSpec.ML_KEM_512)
    kpg.generateKeyPair().also { kp ->
        val f = KeyFactory.getInstance(ML_KEM)
        val privateSpec = f.getKeySpec<PKCS8EncodedKeySpec>(kp.private)

        val pubSpec = f.getKeySpec<X509EncodedKeySpec>(kp.public)
    }
}

val ks = KEM.getInstance(ML_KEM)
val enc = ks.newEncapsulator(kp.public)
val dec = ks.newDecapsulator(kp.private)


enc.encapsulate().also {
    println(enc.secretSize())
}

32


javax.crypto.KEM$Encapsulated@61d6a8e9